<a href="https://colab.research.google.com/github/daylightzjr/daylightzjr/blob/main/transformers%E6%96%B9%E6%A1%88.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers
!pip install datasets
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.1/316.1 kB 17.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 18.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.2 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.6.1
    Uninstalling fsspec-2024.6.1:
   

In [2]:
!pip install torch

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinu

#step1导入相关的包

In [3]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification , TrainingArguments , Trainer
from datasets import load_dataset

#step2加载数据集

In [5]:
dataset = load_dataset("csv", data_files="./ChnSentiCorp_htl_all.csv", split="train")
dataset = dataset.filter(lambda x: x["review"] is not None)
dataset[:10]

Generating train split: 0 examples [00:00, ? examples/s]

Filter:   0%|          | 0/7766 [00:00<?, ? examples/s]

{'label': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 'review': ['距离川沙公路较近,但是公交指示不对,如果是"蔡陆线"的话,会非常麻烦.建议用别的路线.房间较为简单.',
  '商务大床房，房间很大，床有2M宽，整体感觉经济实惠不错!',
  '早餐太差，无论去多少人，那边也不加食品的。酒店应该重视一下这个问题了。房间本身很好。',
  '宾馆在小街道上，不大好找，但还好北京热心同胞很多~宾馆设施跟介绍的差不多，房间很小，确实挺小，但加上低价位因素，还是无超所值的；环境不错，就在小胡同内，安静整洁，暖气好足-_-||。。。呵还有一大优势就是从宾馆出发，步行不到十分钟就可以到梅兰芳故居等等，京味小胡同，北海距离好近呢。总之，不错。推荐给节约消费的自助游朋友~比较划算，附近特色小吃很多~',
  'CBD中心,周围没什么店铺,说5星有点勉强.不知道为什么卫生间没有电吹风',
  '总的来说，这样的酒店配这样的价格还算可以，希望他赶快装修，给我的客人留些好的印象',
  '价格比比较不错的酒店。这次免费升级了，感谢前台服务员。房子还好，地毯是新的，比上次的好些。早餐的人很多要早去些。',
  '不错，在同等档次酒店中应该是值得推荐的！',
  '入住丽晶，感觉很好。因为是新酒店，的确有淡淡的油漆味，房间内较新。房间大小合适，卫生间设备齐全，服务态度也很好。网速可能是提升了吧，感觉还好。有免费的插线板、水果刀。。。等一系列的日常用品免费提供，很细心。晚上的自助餐是每人68元，菜品一般、就算说的过去吧。酒店外有西贝筱面等几家饭馆。西贝的菜实在是量大的惊人，人多在酒店吃就不如去西贝吃了。酒店21：30-24点有广式晚茶，吃过几次，味道正宗，就是点心品种略少。哈。酒店地处青山区，包头分昆区、青山区。相对来说：昆区是最繁华的，青山区次之。像包百等大商场全在昆区，从酒店打车到包百商圈车费12元多。青山区也有一个王府井百货，相对来说比包百商圈的略小，从酒店打车8元多。银河广场离酒店有一段距离，打车9、10元吧，银河广场边上有“科丽珑”24小时营业的超市。酒店一楼有小卖部，周围基本没有超市。这次住正赶上有几家公司在这里开会，有时楼道里略吵，晚上还有几次有人敲错门，理论上与酒店管理没关系吧。希望以后还是尽量让会议团体住在一层，也许这样可以好些吧。

#step3 数据集划分


In [6]:
len(dataset)

7765

In [7]:
datasets = dataset.train_test_split(test_size=0.2)


In [8]:
s1 =len(datasets['train'])
s2 =len(datasets['test'])
s1,s2
datasets

DatasetDict({
    train: Dataset({
        features: ['label', 'review'],
        num_rows: 6212
    })
    test: Dataset({
        features: ['label', 'review'],
        num_rows: 1553
    })
})

#step4数据集的预处理(tokenizer)

In [9]:
import torch
tokenizer = AutoTokenizer.from_pretrained("hfl/rbt3")
def prepare_training(example):
  tokenized_exampled = tokenizer(example["review"], truncation=True,max_length=128)
  tokenized_exampled["label"] = example["label"]
  return tokenized_exampled

tokenized_datasets = datasets.map(prepare_training, batched=True, remove_columns=datasets["train"].column_names)
tokenized_datasets

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/19.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/828 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/110k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/269k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

Map:   0%|          | 0/6212 [00:00<?, ? examples/s]

Map:   0%|          | 0/1553 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 6212
    })
    test: Dataset({
        features: ['label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 1553
    })
})

#step5 创建模型

In [10]:
model = AutoModelForSequenceClassification.from_pretrained("hfl/rbt3")

pytorch_model.bin:   0%|          | 0.00/156M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at hfl/rbt3 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [11]:
model.config

BertConfig {
  "_name_or_path": "hfl/rbt3",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "directionality": "bidi",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 3,
  "output_past": true,
  "pad_token_id": 0,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "pooler_num_fc_layers": 3,
  "pooler_size_per_head": 128,
  "pooler_type": "first_token_transform",
  "position_embedding_type": "absolute",
  "transformers_version": "4.42.4",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 21128
}

#step6 创建评估函数

In [12]:
import evaluate
f1_metric = evaluate.load("f1")
accuracy_metric = evaluate.load("accuracy")

In [17]:
def eval_metric(eval_predict):
    predictions, labels = eval_predict
    predictions = predictions.argmax(axis=-1)
    acc = accuracy_metric.compute(predictions=predictions, references=labels)
    f1 = f1_metric.compute(predictions=predictions, references=labels)
    acc.update(f1)
    return acc


#step7 创建TrainingArguments

In [18]:
train_args = TrainingArguments(output_dir="./checkpoints",      # 输出文件夹
                         per_device_train_batch_size=32,  # 训练时的batch_size
                         per_device_eval_batch_size=32,  # 验证时的batch_size
                         logging_steps=10,          # log 打印的频率
                         evaluation_strategy="epoch",     # 评估策略
                         save_strategy="epoch",        #保存策略
                         save_total_limit=3,         #最大保存数
                         learning_rate=2e-5,         # 学习率
                         weight_decay=0.01,          # weight_decay
                         #gradient_checkpointing=True,
                         optim="adafactor",          #优化器
                         metric_for_best_model="f1",     #设定评估指标
                         load_best_model_at_end=True)     # 训练完成后加载最优模型


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


#step8 创建Trainer


In [19]:
from transformers import DataCollatorWithPadding

# *** 参数冻结 ***
#for name, param in model.bert.named_parameters():
#    param.requires_grad = False

trainer = Trainer(model=model,
          args=train_args,
          tokenizer=tokenizer,
          train_dataset=tokenized_datasets["train"],
          eval_dataset=tokenized_datasets["test"],
          data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
          compute_metrics=eval_metric)

#step9 模型训练

In [20]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.190800,0.290246,0.895686,0.923295
2,0.199800,0.276327,0.897618,0.925106
3,0.161700,0.291029,0.897618,0.925527


TrainOutput(global_step=585, training_loss=0.19649441384861613, metrics={'train_runtime': 109.6908, 'train_samples_per_second': 169.896, 'train_steps_per_second': 5.333, 'total_flos': 312831212046336.0, 'train_loss': 0.19649441384861613, 'epoch': 3.0})

In [21]:
trainer.evaluate(tokenized_datasets["test"])

{'eval_loss': 0.2910293936729431,
 'eval_accuracy': 0.8976175144880876,
 'eval_f1': 0.9255269320843091,
 'eval_runtime': 2.6772,
 'eval_samples_per_second': 580.077,
 'eval_steps_per_second': 18.303,
 'epoch': 3.0}

In [22]:
trainer.predict(tokenized_datasets["test"])

PredictionOutput(predictions=array([[-4.0619555 ,  2.9333181 ],
       [-3.0253544 ,  2.469418  ],
       [-3.6198714 ,  2.5660615 ],
       ...,
       [ 1.2647682 , -0.79384506],
       [-3.3943605 ,  2.5319104 ],
       [-1.3698409 ,  1.2950855 ]], dtype=float32), label_ids=array([1, 1, 1, ..., 0, 1, 0]), metrics={'test_loss': 0.2910293936729431, 'test_accuracy': 0.8976175144880876, 'test_f1': 0.9255269320843091, 'test_runtime': 2.7773, 'test_samples_per_second': 559.181, 'test_steps_per_second': 17.643})

#step10 模型预测

In [23]:
sen = "我觉得这家酒店不错，饭很好吃！"
id2_label = {0: "差评！", 1: "好评！"}
model.eval()
with torch.inference_mode():
    inputs = tokenizer(sen, return_tensors="pt")
    inputs = {k: v.cuda() for k, v in inputs.items()}
    logits = model(**inputs).logits
    pred = torch.argmax(logits, dim=-1)
    print(f"输入：{sen}\n模型预测结果:{id2_label.get(pred.item())}")

输入：我觉得这家酒店不错，饭很好吃！
模型预测结果:好评！


In [27]:
from transformers import pipeline

model.config.id2label = id2_label
pipe = pipeline("text-classification", model=model, tokenizer=tokenizer, device=0)

In [26]:
sen = "这个酒店不干净"

In [28]:
pipe(sen)

[{'label': '差评！', 'score': 0.9320176243782043}]